In [1]:
#!pip3 install deepface
#!pip3 install tf-keras
from deepface import DeepFace
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np
import os
import pickle

In [2]:
# Load dataset and extract features
def load_dataset(dataset_path):
    X, y = [], []

    for person_name in os.listdir(dataset_path):
        person_folder = os.path.join(dataset_path, person_name)
        if not os.path.isdir(person_folder):
            continue

        for img_name in os.listdir(person_folder):
            img_path = os.path.join(person_folder, img_name)
            
            # Extract face embedding
            try:
                face = DeepFace.represent(img_path, model_name="Facenet512", detector_backend="mtcnn")
                if face:  # Check if the embedding is not empty
                    # Extract the embedding from the first element of the list
                    embedding = face[0]["embedding"]
                    X.append(np.array(embedding))
                    y.append(person_name)
            except Exception as e:
                print(f"Error processing image {img_path}: {e}")
                continue

        print('Finished loading examples for class:', person_name)
    return np.array(X), np.array(y)

In [ ]:
# Load dataset
dataset_path = "dataset/"
X, y = load_dataset(dataset_path)

In [ ]:
with open("facenet_data.pkl", "wb") as file:
    pickle.dump((X, y), file)

print("Data saved")

In [ ]:
with open("facenet_data.pkl", "rb") as file:
    X, y = pickle.load(file)

In [ ]:
# Train KNN
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
print("X_train: ",X_train.shape)
print("X_test: ",X_test.shape)
print("y_train: ",y_train.shape)
print("y_test: ",y_test.shape)

In [ ]:
pca = PCA(n_components=150, svd_solver="randomized", whiten=True)
svm = SVC(C=1000, gamma=0.05, class_weight='balanced', kernel='linear', random_state=42)
model = make_pipeline(pca, svm)

svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)

In [ ]:
svm_train_acc = accuracy_score(y_train, svm.predict(X_train))
svm_test_acc = accuracy_score(y_test, y_pred)

print(f"Training accuracy of SVM is : {svm_train_acc}")
print(f"Test accuracy of SVM is : {svm_test_acc}")

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))